In [29]:
%matplotlib inline

# import basic libraries
import yt
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u

# import halo catalogue func
from yt.analysis_modules.halo_analysis.api import *

In [14]:
import tempfile
import shutil
import os

# Create temporary directory for storing files
tmpdir = tempfile.mkdtemp()

In [43]:
# load dataset
ds = yt.load('~/../../tigress/cen/LG4_2048_40pc/RD0070/redshift0070')

yt : [INFO     ] 2017-06-26 16:51:19,043 Parameters: current_time              = 158.01757996843
yt : [INFO     ] 2017-06-26 16:51:19,046 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2017-06-26 16:51:19,049 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-06-26 16:51:19,053 Parameters: domain_right_edge         = [ 1.  1.  1.]
yt : [INFO     ] 2017-06-26 16:51:19,056 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2017-06-26 16:51:19,058 Parameters: current_redshift          = 4.9999999832278
yt : [INFO     ] 2017-06-26 16:51:19,060 Parameters: omega_lambda              = 0.712
yt : [INFO     ] 2017-06-26 16:51:19,062 Parameters: omega_matter              = 0.288
yt : [INFO     ] 2017-06-26 16:51:19,064 Parameters: hubble_constant           = 0.6933


In [16]:
ds.print_stats()

Parsing Hierarchy : 100%|██████████| 44224/44224 [00:05<00:00, 7899.39it/s] 
yt : [INFO     ] 2017-06-26 16:07:12,103 Gathering a field list (this may take a moment.)


level	# grids	       # cells	     # cells^3
----------------------------------------------
  0	   256	       2097152	           128
  1	    60	        547304	            82
  2	   187	       2071552	           128
  3	   898	       9903616	           215
  4	  5193	      55377920	           382
  5	  9876	      53324192	           377
  6	  9632	      37417048	           335
  7	  7854	      22891480	           284
  8	  5388	      12002584	           229
  9	  1830	       6173384	           184
 10	  1321	       6643200	           188
 11	  1729	       9688816	           214
----------------------------------------------
   	 44224	     218138248


t = 2.07434497e+02 = 4.93026195e+16 s = 1.56230573e+09 years

Smallest Cell:
	Width: 3.301e-05 Mpc
	Width: 3.301e+01 pc
	Width: 6.810e+06 AU
	Width: 1.019e+20 cm


In [17]:
ds.field_list

[('all', 'creation_time'),
 ('all', 'dynamical_time'),
 ('all', 'initial_mass'),
 ('all', 'metallicity_fraction'),
 ('all', 'particle_index'),
 ('all', 'particle_mass'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'particle_type'),
 ('all', 'particle_velocity_x'),
 ('all', 'particle_velocity_y'),
 ('all', 'particle_velocity_z'),
 ('all', 'typeia_fraction'),
 ('enzo', 'Dark_Matter_Density'),
 ('enzo', 'Density'),
 ('enzo', 'Electron_Density'),
 ('enzo', 'H2II_Density'),
 ('enzo', 'H2I_Density'),
 ('enzo', 'HII_Density'),
 ('enzo', 'HI_Density'),
 ('enzo', 'HM_Density'),
 ('enzo', 'HeIII_Density'),
 ('enzo', 'HeII_Density'),
 ('enzo', 'HeI_Density'),
 ('enzo', 'Metal_Density'),
 ('enzo', 'Temperature'),
 ('enzo', 'TotalEnergy'),
 ('enzo', 'x-velocity'),
 ('enzo', 'y-velocity'),
 ('enzo', 'z-velocity'),
 ('io', 'creation_time'),
 ('io', 'dynamical_time'),
 ('io', 'initial_mass'),
 ('io', 'metallicity_fraction'),
 ('io', 'part

In [26]:
# specify boundaries of zoom-in box
# scaling factor multiplied by info from text file 
# units in cm
xmin = 0.39319589
ymin = 0.42984636
zmin = 0.41706725

xmax = 0.56298484
ymax = 0.55089246
zmax = 0.56698254

In [42]:
# limit dataset
obj = ds.arbitrary_grid([xmin, ymin, zmin], [xmax, ymax, zmax], dims=[128, 128, 128])

TypeError: __init__() missing 1 required positional argument: 'dims'

In [35]:
# calculate center of sphere
xcent = xmin + (xmax-xmin)/2
ycent = ymin + (ymax-ymin)/2
zcent = zmin + (zmax-zmin)/2

#calculate radius of sphere
dist = ((xmax-xmin)**2 + (ymax-ymin)**2 + (zmax-zmin)**2)**0.5
radius = dist/2.
radius = 2.22535525e+25*radius * u.cm

In [40]:
sp = ds.sphere([xcent, ycent, zcent], (radius.value, 'cm'))

In [18]:
# load halo dataset
halos_ds = yt.load('./LG4_2048_40pc/rd0070/halo_catalogs/catalog/catalog0070_thres160.0.h5')

yt : [INFO     ] 2017-06-26 16:07:30,143 Parameters: current_time              = 3.75573046409e+16
yt : [INFO     ] 2017-06-26 16:07:30,144 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2017-06-26 16:07:30,146 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-06-26 16:07:30,147 Parameters: domain_right_edge         = [  2.22535525e+25   2.22535525e+25   2.22535525e+25]
yt : [INFO     ] 2017-06-26 16:07:30,148 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2017-06-26 16:07:30,149 Parameters: current_redshift          = 4.99999998323
yt : [INFO     ] 2017-06-26 16:07:30,150 Parameters: omega_lambda              = 0.712
yt : [INFO     ] 2017-06-26 16:07:30,151 Parameters: omega_matter              = 0.288
yt : [INFO     ] 2017-06-26 16:07:30,151 Parameters: hubble_constant           = 0.6933


In [19]:
halos_ds.field_list

yt : [INFO     ] 2017-06-26 16:07:30,159 Allocating for 9.178e+03 particles (index particle type 'all')
yt : [INFO     ] 2017-06-26 16:07:30,166 Identified 3.025e+03 octs


[('all', 'particle_identifier'),
 ('all', 'particle_mass'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'virial_radius'),
 ('halos', 'particle_identifier'),
 ('halos', 'particle_mass'),
 ('halos', 'particle_position_x'),
 ('halos', 'particle_position_y'),
 ('halos', 'particle_position_z'),
 ('halos', 'virial_radius')]

In [47]:
# Instantiate a catalog using those two paramter files
hc = HaloCatalog(data_ds=obj, halos_ds=halos_ds, output_dir=os.path.join(tmpdir, 'halo_catalog'))
hc.load()

In [33]:
# create projection plot
p = yt.ProjectionPlot(sp, "x", "Dark_Matter_Density", width=(1.5, 'Mpc'))
p.set_zlim(field="Dark_Matter_Density", zmin=1e23, zmax=1e27)
#p.annotate_halos(hc, factor = "particle_mass")
p.annotate_halos(hc)
p.display()

AttributeError: 'YTSphere' object has no attribute 'coordinates'

In [22]:
obj

YTArbitraryGrid (redshift0090): , left_edge=[  1.05000065e+25   1.14787303e+25   1.11374736e+25] cm, right_edge=[  1.50340954e+25   1.47111773e+25   1.51408510e+25] cm, ActiveDimensions=[128 128 128]

In [41]:
sp

YTSphere (redshift0090): , center=[  1.27670510e+25   1.30949538e+25   1.31391623e+25] cm, radius=2.8575467812729705e+24 cm